# Data treatment and Neural Network Training using IPMA data
João Oliveira and Edgar Mendes

In [4]:
#-----------------------Imports

import os
import requests
import json
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import io
from io import BytesIO
from PIL import Image, ImageDraw
import pytz
import numpy as np

#-----------------------Constantes

DADOS_ULTIMAS_3_HORAS = "https://api.ipma.pt/open-data/observation/meteorology/stations/obs-surface.geojson" #por exemplo se forem 9PM tem os dados entre 5PM e 7PM de hora a hora para todas as estações
DADOS_IDS_ESTACOES = "https://api.ipma.pt/open-data/observation/meteorology/stations/stations.json"
local_tz = pytz.timezone('Europe/Lisbon') #Define o fuso horário local

#-------Recorte para cada distrito

boxVianaDoCastelo = (570, 428, 770, 628)
boxLeiria = (574, 902, 774, 1102)
boxAveiro = (660, 679, 820, 839)
boxBeja = (704, 1332, 944, 1572)
boxBraga = (686, 477, 846, 637)
boxBraganca = (931, 458, 1161, 688)
boxCasteloBranco = (792, 833, 1022, 1063)
boxPortalegre = (773, 1021, 1003, 1251)
boxPorto = (686, 571, 846, 731)
boxSantarem = (625, 997, 855, 1227)
boxCoimbra = (645, 792, 845, 992)
boxEvora = (740, 1183, 940, 1383)
boxFaro = (736, 1546, 936, 1746)
boxGuarda = (859, 712, 1059, 912)
boxLisboa = (513, 1149, 713, 1349)
boxSetubal = (559, 1193, 759, 1393)
boxVilaReal = (770, 527, 970, 727)
boxViseu = (740, 682, 940, 882)

#-------IDs das estações

idVianaDoCastelo = 1240610
idLeiria = 1210718
idAveiro = 1210702
idBeja = 1200562
idBraga = 6212124
idBraganca = 1200575
idCasteloBranco = 1200570
idPortalegre = 1200571
idPorto = 1240903
idSantarem = 1210734
idCoimbra = 1210707
idEvora = 1200558
idFaro = 1200554
idGuarda = 1210683
idLisboa = 7240919
idSetubal = 1210770
idVilaReal = 1240566
idViseu = 1240675

ids = np.array([1240610, 1210718, 1210702, 1200562, 6212124, 1200575, 1200570, 1200571, 
                1240903, 1210734, 1210707, 1200558, 1200554, 1210683, 7240919, 1210770, 
                1240566, 1240675])

station_box_dict = {idVianaDoCastelo: boxVianaDoCastelo,
                    idLeiria: boxLeiria,
                    idAveiro: boxAveiro,
                    idBeja: boxBeja,
                    idBraga: boxBraga,
                    idBraganca: boxBraganca,
                    idCasteloBranco: boxCasteloBranco,
                    idPortalegre: boxPortalegre,
                    idPorto: boxPorto,
                    idSantarem: boxSantarem,
                    idCoimbra: boxCoimbra,
                    idEvora: boxEvora,
                    idFaro: boxFaro,
                    idGuarda: boxGuarda,
                    idLisboa: boxLisboa,
                    idSetubal: boxSetubal,
                    idVilaReal: boxVilaReal,
                    idViseu: boxViseu}


#-----------------------Funções

def get_data(url):
    response = requests.get(f"{url}")
    if response.status_code == 200:
        print("Sucessfully fetched the data!") 
        return response.json() #https://www.educative.io/answers/how-to-make-api-calls-in-python
    else:
        print(f"Hello there, there's a {response.status_code} error with your request.")
        
def normalize_precipitation_value(precipitation_value):
    return int(round((precipitation_value/240)*100,0)) #o valor normalizado ta a ser arredondado pq as pastas sao de valores inteiros. https://www.ipma.pt/pt/oclima/extremos.clima/ Vou usar o valor máximo aqui como referencia

def get_images_and_data_from_ipma():
    data = get_data(DADOS_ULTIMAS_3_HORAS)
    # Faz a requisição à apiEstacoes e à apiMeteo para obter as informações meteorológicas 
    final_result = {}
    for feature in data['features']:
        if feature['properties']['idEstacao']  in ids:
            station_data = feature['properties']
            id_estacao = station_data['idEstacao']
            # Converte a hora da estação para o fuso horário local
            date_time_utc = datetime.fromisoformat(station_data['time'])
            date_time = datetime.fromisoformat(station_data['time']).replace(tzinfo=pytz.utc).astimezone(local_tz)
            date_str, hour_str = date_time.strftime('%Y-%m-%d %H:%M').split()
            precipitation = station_data['precAcumulada']

            if date_str not in final_result:
                final_result[date_str] = {hour_str: precipitation}
            else:
                final_result[date_str][hour_str] = normalize_precipitation_value(max(0, precipitation))

            url_image = f"https://www.ipma.pt/resources.www/transf/radar/por/pcr-{date_time_utc.strftime('%Y-%m-%d')}T{date_time_utc.strftime('%H%M')}.png"
            response = requests.get(url_image)
            image_data = io.BytesIO(response.content)
            image = Image.open(image_data)
            print(f"{url_image} ({hour_str}h)")
            # remover os pixeis pretos 
            image = remove_black_pixels(image)

            # Corte da imagem
            # verifica se a pasta id_estacao existe e cria se não existir
            if not os.path.exists(f"dataset/images/{id_estacao}"):
                os.makedirs(f"dataset/images/{id_estacao}")

            # verifica se a pasta date_str existe e cria se não existir
            if not os.path.exists(f"dataset/images/{id_estacao}/{date_str}"):
                print(id_estacao)
                os.makedirs(f"dataset/images/{id_estacao}/{date_str}")

            region = image.crop(station_box_dict[id_estacao])
            region.save(f"dataset/images/{id_estacao}/{date_str}/{date_time.strftime('%Y-%m-%dT%H%M')}.png")
            region.close()

            # Verifica se o arquivo JSON para esta estação já existe, se não, cria o arquivo
            filename = f"dataset/precipitation/{id_estacao}.json"
            if not os.path.isfile(filename):
                with open(filename, 'w') as file:
                    json.dump({}, file)

            # Carrega o conteúdo do arquivo JSON para a variável "precipitation_data"
            with open(filename, 'r') as file:
                precipitation_data = json.load(file)

            # Adiciona as informações meteorológicas ao arquivo JSON
            for date in final_result:
                if date not in precipitation_data:
                    precipitation_data[date] = final_result[date]
                else:
                    precipitation_data[date].update(final_result[date])

            # Escreve o conteúdo atualizado no arquivo JSON
            with open(filename, 'w') as file:
                json.dump(precipitation_data, file, indent=4)

    print("Dados atualizados com sucesso!")
    
def dict_to_array2D(input_dict):
    # Create an empty list to hold the values
    output_array = np.empty(0)
    # Loop over each key-value pair in the dictionary
    for date_dict in input_dict.values():
        for value in date_dict.values():
            # Append the value to the output array
            output_array = np.append(output_array,value)
    # Reshape to 2D array
    output_array = np.reshape(output_array, (-1, 1))
    # Return the output array
    return output_array

def remove_black_pixels(image):
    # Convert the image to RGBA mode (if it's not already in RGBA mode)
    image = image.convert("RGBA")
    # Get the pixel data as a list of tuples
    pixels = list(image.getdata())
    # Replace every black pixel with transparent
    new_pixels = []
    for pixel in pixels:
        if pixel[0] == 0 and pixel[1] == 0 and pixel[2] == 0:
            new_pixels.append((0, 0, 0, 0))
        else:
            new_pixels.append(pixel)

    # Create a new image with the same size and mode as the original image
    new_image = Image.new(image.mode, image.size)
    # Update the new image with the new pixel data
    new_image.putdata(new_pixels)
    # Return the new image
    return new_image

In [1]:
#--------Criação das pastas de 0 a 100

for i in range(101):
    os.mkdir(str(i)) #---Cria as pastas do 0 ao 100
print("Pastas criadas!")

Pastas criadas!


#  API Precipition Information
Observação Meteorológica de Estações, últimas 3 horas (formato GeoJSON) com as imagens (png)

In [5]:
get_images_and_data_from_ipma()
#<class 'PIL.PngImagePlugin.PngImageFile'> | Image: <PIL.PngImagePlugin.PngImageFile image mode=RGBA size=1500x2331 at 0x1DADDEBEF40>


Sucessfully fetched the data!
https://www.ipma.pt/resources.www/transf/radar/por/pcr-2023-05-03T1200.png (13:00h)
https://www.ipma.pt/resources.www/transf/radar/por/pcr-2023-05-03T1200.png (13:00h)
https://www.ipma.pt/resources.www/transf/radar/por/pcr-2023-05-03T1200.png (13:00h)
https://www.ipma.pt/resources.www/transf/radar/por/pcr-2023-05-03T1200.png (13:00h)
https://www.ipma.pt/resources.www/transf/radar/por/pcr-2023-05-03T1200.png (13:00h)
https://www.ipma.pt/resources.www/transf/radar/por/pcr-2023-05-03T1200.png (13:00h)
https://www.ipma.pt/resources.www/transf/radar/por/pcr-2023-05-03T1200.png (13:00h)
https://www.ipma.pt/resources.www/transf/radar/por/pcr-2023-05-03T1200.png (13:00h)
https://www.ipma.pt/resources.www/transf/radar/por/pcr-2023-05-03T1200.png (13:00h)
https://www.ipma.pt/resources.www/transf/radar/por/pcr-2023-05-03T1200.png (13:00h)
https://www.ipma.pt/resources.www/transf/radar/por/pcr-2023-05-03T1200.png (13:00h)
https://www.ipma.pt/resources.www/transf/radar

# Model Training


In [4]:
#-----------------------Imports

import numpy as np
import io

from sklearn.model_selection import train_test_split

import tensorflow as tf
import tensorflow.keras as keras

from keras import layers
from keras import callbacks
from keras.models import Sequential
from keras.layers import Conv2D, Dense, Flatten, Dropout, MaxPooling2D
from keras.utils.vis_utils import plot_model

import pydot



#-----------------------Constantes

N_CHANNELS = 4
N_NEURONIOS = 32
FILTER_SIZE = 3
MAX_POOL_SIZE = (3,3)
N_CLASSES = 100
N_EPOCHS = 200
N_STRIDES = 2
DROPOUT_VALUE = 0.5
IMAGE_WIDTH, IMAGE_HEIGHT = 200,200

In [5]:
images_array = np.empty((0, IMAGE_WIDTH, IMAGE_HEIGHT, 4))
data_array = np.empty(0)
# Open the JSON file
with open('dataset/precipitation/1240610.json') as f:
    # Load the JSON data
    data = json.load(f)

data_array = dict_to_array2D(data)

#-----------------------------------------

images_folder_path = 'dataset/images/1240610/'

# Get a list of all the files in the current folder
file_list = os.listdir(images_folder_path)

# Filter the list to only include folders files
days_folders_list = [file for file in file_list if os.path.isdir(os.path.join(images_folder_path, file))]

for day_folder in days_folders_list:
    current_path = images_folder_path + day_folder
    #Get a list of all files in the folder 
    files_list = os.listdir(current_path)
    # Filter the list to only include images files
    image_list = [file for file in files_list if file.endswith('.png')]
    for i in range(len(image_list)):
        image_path = os.path.join(current_path, image_list[i])
        image = Image.open(image_path)
        img_np = np.array(image)
        images_array = np.append(images_array, [img_np], axis=0)
        
train_images, val_images, train_values, val_values = train_test_split(images_array, data_array, test_size=0.2, random_state=42)

train_images = tf.keras.utils.normalize(train_images, axis=1)
val_images = tf.keras.utils.normalize(val_images, axis=1)

In [6]:
#Ver quantos valores há

def flatten_array(input_array):
    output_array = np.reshape(input_array, (-1,))
    return output_array

def count_values(arr):
    counts = {}
    for num in arr:
        if num in counts:
            counts[num] += 1
        else:
            counts[num] = 1
    return counts

counts = count_values(flatten_array(data_array))
print(counts)

{0.0: 51, 1.0: 2, 5.0: 1}


In [8]:
model = tf.keras.models.Sequential()
model.add(layers.Conv2D(N_NEURONIOS, MAX_POOL_SIZE, activation='relu', input_shape=(IMAGE_WIDTH, IMAGE_HEIGHT, N_CHANNELS)))
model.add(tf.keras.layers.Conv2D(N_NEURONIOS, FILTER_SIZE, strides=N_STRIDES, padding='same', activation='relu'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.MaxPooling2D(pool_size=MAX_POOL_SIZE))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Conv2D(N_NEURONIOS*2, FILTER_SIZE, padding='same', activation='relu'))
model.add(tf.keras.layers.BatchNormalization())
# model.add(tf.keras.layers.MaxPooling2D(pool_size=MAX_POOL_SIZE))
# model.add(tf.keras.layers.BatchNormalization())
# model.add(tf.keras.layers.Conv2D(N_NEURONIOS*4, FILTER_SIZE, padding='same', activation='relu'))
# model.add(tf.keras.layers.BatchNormalization())
# model.add(tf.keras.layers.MaxPooling2D(pool_size=MAX_POOL_SIZE))
# model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(N_NEURONIOS*8, activation='relu'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Dropout(DROPOUT_VALUE))
model.add(tf.keras.layers.Dense(N_CLASSES, activation='softmax'))

#model.summary()
#plot_model(model, show_shapes=True, show_layer_names=True, to_file='model.png')

model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

In [9]:
history = model.fit(train_images, train_values, epochs= N_EPOCHS, validation_data=(val_images, val_values))

Epoch 1/200
2/2 [==============================] - 8s 2s/step - loss: 5.5871 - accuracy: 0.0698 - val_loss: 4.7123 - val_accuracy: 0.0000e+00
Epoch 2/200
2/2 [==============================] - 3s 1s/step - loss: 5.5132 - accuracy: 0.2326 - val_loss: 4.5764 - val_accuracy: 0.0909
Epoch 3/200
2/2 [==============================] - 2s 611ms/step - loss: 4.7485 - accuracy: 0.2326 - val_loss: 4.5578 - val_accuracy: 0.0909
Epoch 4/200
2/2 [==============================] - 2s 590ms/step - loss: 4.8112 - accuracy: 0.3023 - val_loss: 4.5396 - val_accuracy: 0.0909
Epoch 5/200
2/2 [==============================] - 2s 836ms/step - loss: 4.5820 - accuracy: 0.2326 - val_loss: 4.4738 - val_accuracy: 0.0909
Epoch 6/200
2/2 [==============================] - 2s 780ms/step - loss: 4.4469 - accuracy: 0.3721 - val_loss: 4.4457 - val_accuracy: 0.0909
Epoch 7/200
2/2 [==============================] - 2s 686ms/step - loss: 4.3673 - accuracy: 0.3721 - val_loss: 4.4216 - val_accuracy: 0.0909
Epoch 8/200
2/2

Epoch 59/200
2/2 [==============================] - 2s 759ms/step - loss: 0.6632 - accuracy: 0.9070 - val_loss: 0.7094 - val_accuracy: 0.9091
Epoch 60/200
2/2 [==============================] - 2s 757ms/step - loss: 0.7599 - accuracy: 0.9070 - val_loss: 0.7021 - val_accuracy: 0.9091
Epoch 61/200
2/2 [==============================] - 2s 791ms/step - loss: 1.0577 - accuracy: 0.8837 - val_loss: 0.6999 - val_accuracy: 0.9091
Epoch 62/200
2/2 [==============================] - 3s 1s/step - loss: 0.8952 - accuracy: 0.9070 - val_loss: 0.6987 - val_accuracy: 0.9091
Epoch 63/200
2/2 [==============================] - 3s 1s/step - loss: 0.5767 - accuracy: 0.9767 - val_loss: 0.6939 - val_accuracy: 0.9091
Epoch 64/200
2/2 [==============================] - 2s 929ms/step - loss: 0.7481 - accuracy: 0.8837 - val_loss: 0.6840 - val_accuracy: 0.9091
Epoch 65/200
2/2 [==============================] - 2s 869ms/step - loss: 0.7525 - accuracy: 0.8605 - val_loss: 0.6757 - val_accuracy: 0.9091
Epoch 66/200

2/2 [==============================] - 2s 764ms/step - loss: 0.0199 - accuracy: 1.0000 - val_loss: 0.7117 - val_accuracy: 0.9091
Epoch 174/200
2/2 [==============================] - 2s 751ms/step - loss: 0.1256 - accuracy: 1.0000 - val_loss: 0.7144 - val_accuracy: 0.9091
Epoch 175/200
2/2 [==============================] - 2s 740ms/step - loss: 0.2701 - accuracy: 0.9767 - val_loss: 0.7176 - val_accuracy: 0.9091
Epoch 176/200
2/2 [==============================] - 2s 746ms/step - loss: 0.1721 - accuracy: 0.9767 - val_loss: 0.7221 - val_accuracy: 0.9091
Epoch 177/200
2/2 [==============================] - 2s 774ms/step - loss: 0.2011 - accuracy: 0.9767 - val_loss: 0.7203 - val_accuracy: 0.9091
Epoch 178/200
2/2 [==============================] - 2s 771ms/step - loss: 0.0452 - accuracy: 1.0000 - val_loss: 0.7136 - val_accuracy: 0.9091
Epoch 179/200
2/2 [==============================] - 2s 791ms/step - loss: 0.0513 - accuracy: 1.0000 - val_loss: 0.7017 - val_accuracy: 0.9091
Epoch 180/200